# Analyzing scaling with mass 
October 13, 2021 
Venkitesh Ayyar

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
%matplotlib widget

In [3]:
def f_get_site(x,y,L):
    return x+y*L

def f_get_base(xc,yc,L,quad):
    
    if quad==1: 
        xb=block_x*xc; yb=block_y*yc
    elif quad==2: 
        xb=(block_x*xc-1+L)%L; yb=block_y*yc
    elif quad==3: 
        xb=(block_x*xc-1+L)%L; yb=(block_y*yc-1+L)%L
    elif quad==4: 
        xb=block_x*xc; yb=(block_y*yc-1+L)%L        
    
    return xb,yb

def f_conv_complex(strg):
    ''' Convert input from a+ib to python complex form'''
    arr=[np.float64(k) for k in strg.split('+i')]
    return complex(arr[0],arr[1])
# f_get_base(2,2,L,quad=4)


In [19]:
# fname='../3_complex_scalar_adaptivemgrid/1_regular_tl_code/Near-null_L32_blk2_ndof2.txt'
fname='../3_complex_scalar_adaptivemgrid/2_nontl_code/Near-null_L32_blk2_ndof2.txt'


# df=pd.read_csv(fname,delimiter=',',names=cols,index_col=False)
a1=np.loadtxt(fname,delimiter='\n',dtype=str)

In [20]:
arr=np.loadtxt(fname,delimiter='\n',dtype=str)
arr1=np.vectorize(f_conv_complex)(arr[:]) # Skip last element (null string) and first (iteration)

In [21]:
arr1.shape

(3392,)

In [22]:
L=32;levels=4
block_x=2; block_y=2


In [23]:
# Build list of n_dof for each level

nc_list=[1,2,2,2,2]
# Build list of lattice size
L_list=[(L//(block_x**i))**2 for i in range(levels)]

print(nc_list,L_list)


size_lst=[]
for i in range(levels):
    print(L_list[i],nc_list[i],L_list[i]*nc_list[i+1]*nc_list[i])
    size_lst.append(L_list[i]*nc_list[i+1]*nc_list[i])
    
print(size_lst)
print(sum(size_lst))

[1, 2, 2, 2, 2] [1024, 256, 64, 16]
1024 1 2048
256 2 1024
64 2 256
16 2 64
[2048, 1024, 256, 64]
3392


In [24]:
# 32**2 * (2*1) + 16**2 * (2*2) + 8**2 * (2*2) + 4**2 * (2*2) 

In [25]:
assert sum(size_lst)==arr1.shape[0], "Sizes unequal %d %d "%(sum(size_lst),arr1.shape[0])

In [42]:
nn_lst=[]
start_idx=0
for i,size in zip(range(0,levels),size_lst):
    arr=arr1[start_idx:start_idx+size].reshape(L_list[i],nc_list[i+1],nc_list[i])
    print(arr.shape)
    nn_lst.append(arr)
    start_idx+=size
    print(start_idx,size)

(1024, 2, 1)
2048 2048
(256, 2, 2)
3072 1024
(64, 2, 2)
3328 256
(16, 2, 2)
3392 64


In [52]:
# Check block normalization
for idx in range(0,levels):
    Lf=L//(block_x**idx)
    Lc=Lf//block_x
    print(idx,Lf,Lc)

    for i in range(nc_list[idx]):
        for xc in range(Lc): 
            for yc in range(Lc):
                xb,yb=f_get_base(xc,yc,Lf,quad=4)

                # print(xb,yb,end='\t')
                # print(xc*block_x,yc*block_y,end='\t')
                # site=[((xb+x1+L)%L,(yb+y1+L)%L) for x1 in range(block_x) for y1 in range(block_y)]
                # print(site)

                site=[f_get_site((xb+x1+Lf)%Lf,(yb+y1+Lf)%Lf,Lf) for x1 in range(block_x) for y1 in range(block_y)]

                # print(site)
                print(nn_lst[idx][site,i,:].shape)
                norm=np.linalg.norm(np.abs(nn_lst[idx][site,i,:]))
    # #             # print(norm)

                if ~np.isclose(norm,1.0,rtol=1e-20,atol=1e-14): 
                    print("Not normalized",norm)


0 32 16
(4, 1)
Not normalized 1.0325721124340725
(4, 1)
Not normalized 1.015550328379592
(4, 1)
Not normalized 1.0126326820616607
(4, 1)
Not normalized 0.9844758165234727
(4, 1)
Not normalized 1.0292223672645477
(4, 1)
Not normalized 1.01056110057031
(4, 1)
Not normalized 1.0116633447906163
(4, 1)
Not normalized 1.0280988106399205
(4, 1)
Not normalized 1.0163906330643768
(4, 1)
Not normalized 1.006733580766066
(4, 1)
Not normalized 1.0149493821734408
(4, 1)
Not normalized 1.0318448164731964
(4, 1)
Not normalized 1.054387903810387
(4, 1)
Not normalized 1.0438068049536644
(4, 1)
Not normalized 1.0217897611101872
(4, 1)
Not normalized 1.02793827472447
(4, 1)
Not normalized 1.0227690400122553
(4, 1)
Not normalized 0.9892665160210329
(4, 1)
Not normalized 1.0016037669749298
(4, 1)
Not normalized 0.9690264322501598
(4, 1)
Not normalized 1.017277324804961
(4, 1)
Not normalized 1.0227742286231363
(4, 1)
Not normalized 1.0011763105839722
(4, 1)
Not normalized 1.019737948231653
(4, 1)
Not normal

In [ ]:
# # Check block normalization
# for idx in range(0,levels):
#     Lf=L//(block_x**idx)
#     Lc=Lf//block_x
#     print(idx,Lf,Lc)

#     for xc in range(Lc): 
#         for yc in range(Lc):
#             xb,yb=f_get_base(xc,yc,Lf,quad=4)
            
#             # print(xb,yb,end='\t')
#             # print(xc*block_x,yc*block_y,end='\t')
#             # site=[((xb+x1+L)%L,(yb+y1+L)%L) for x1 in range(block_x) for y1 in range(block_y)]
#             # print(site)
            
#             site=[f_get_site((xb+x1+Lf)%Lf,(yb+y1+Lf)%Lf,Lf) for x1 in range(block_x) for y1 in range(block_y)]

#             # print(site)
#             norm=np.linalg.norm(np.abs(nn_lst[idx][site]))
# # #             # print(norm)
            
#             if ~np.isclose(norm,1.0,rtol=1e-20,atol=1e-14): 
#                 print("Not normalized",norm)



In [53]:
for i in nn_lst:
    print(i.shape)

(1024, 2, 1)
(256, 2, 2)
(64, 2, 2)
(16, 2, 2)
